In [30]:
import pandas as pd
import numpy as np
from scipy import stats

## データの読み込み、準備

In [3]:
url = 'http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv'
df = pd.read_csv(url)

In [4]:
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


女性向けデータの削除、介入を表す列（treatment）の追加

In [8]:
male_df = df.loc[df.segment != 'Womens E-Mail'].copy()
male_df['treatment'] = 0
male_df.loc[male_df.segment == 'Mens E-Mail', 'treatment'] = 1

介入の有無に応じた集計を確認

In [28]:
summary_by_segment = pd.merge(
                        male_df.groupby('treatment')[['conversion', 'spend']].mean().reset_index(), 
                        male_df.groupby('treatment').size().reset_index(),
                        on='treatment'
                    ).rename(columns = {'conversion': 'conversion_mean', 'spend': 'spend_mean', 0:'count'})
print(summary_by_segment)

   treatment  conversion_mean  spend_mean  count
0          0         0.005726    0.652789  21306
1          1         0.012531    1.422617  21307


## 有意差検定

In [36]:
mens_mail = np.array(male_df.loc[male_df['treatment'] == 1]['spend'])
no_mail = np.array(male_df.loc[male_df['treatment'] == 0]['spend'])

対応の無いデータに対するt検定。Welchのt検定を行うので、ttest_indメソッドでequal_val=Falseとする

In [45]:
stats.ttest_ind(mens_mail, no_mail, equal_var=False)

Ttest_indResult(statistic=5.300140358411668, pvalue=1.1638149682254859e-07)